Detección de caras con webcam con detectores de OpenCV

In [2]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN

# Models load and setup
# VJ
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
# DNN
dnn_model = "deploy.prototxt.txt"
dnn_weights = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(dnn_model, dnn_weights)
# MTCNN
mtcnn_det = MTCNN()

# Webcam connection
video_capture = cv2.VideoCapture(0)

# Detector index
det = 0
while True:
    ret, frame = video_capture.read()    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Depending on the active detector
    if det == 0:
        # VJ
        faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Draw bounding boxes
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    elif det == 1:
        # DNN
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104.0, 177.0, 123.0], False, False)
        net.setInput(blob)
        detections = net.forward()

        # Draw bounding boxes
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x2, y2) = box.astype("int")
                cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
    else: #MTCNN
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mtcnn_det.detect_faces(rgb_frame)

        # Draw bounding boxes
        for result in results:
            x, y, width, height = result['box']
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255), 2)
    
    # Shgow results
    cv2.imshow('Face Detection', frame)
    
    # 'd' to change detector, Esc to finish
    key = cv2.waitKey(1) & 0xFF
    if key == ord('d'):
        det += 1
        if det > 2:
            det = 0
    elif key == 27:
        break

# Close windoews and release camera
video_capture.release()
cv2.destroyAllWindows()


